In [1]:
!pip3 install tqdm

    100% |████████████████████████████████| 71kB 10.4MB/s ta 0:00:01
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import clean_data
import os
import pandas as pd
import numpy as np

from sklearn import model_selection

import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import get_execution_role

session = sagemaker.Session()

raw_data = pd.read_csv('00_raw_data.csv')
raw_data.drop(columns = {'Unnamed: 0'}, inplace = True)

In [3]:
preproc = clean_data.Preprocess(raw_data, 'train')
df, X = preproc()

100%|██████████| 7998/7998 [32:16<00:00,  4.13it/s]


time: 2206.198136329651


In [4]:
df.to_csv('usme.csv')

In [5]:
# remove score before pca and put it back in later at the first position

scores = df.score
df.drop(columns = {'score'}, inplace = True)
from sklearn.decomposition import PCA
components = 7
pca = PCA(n_components=components)
pca_x = pca.fit_transform(X)

data_dir = 'data_for_model'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
train = pd.concat([scores, pd.DataFrame(pca_x)], axis = 1)

train.to_csv(os.path.join(data_dir, 'train.csv'), header = None, index = None)

prefix = 'landis-score-predict'
train_location = session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix = prefix)

In [6]:
from time import gmtime, strftime
container = get_image_uri(session.boto_region_name, 'xgboost')
role = get_execution_role()
training_params = {}

training_params['RoleArn'] = role

training_params['AlgorithmSpecification'] = {
    'TrainingImage' : container,
    'TrainingInputMode' : 'File'
}

training_params['OutputDataConfig'] = {
    'S3OutputPath' : 's3://' + session.default_bucket() + '/' + prefix + '/output'
}

training_params['ResourceConfig'] = {
    'InstanceCount' : 1,
    'InstanceType' : 'ml.m4.10xlarge',
    'VolumeSizeInGB' : 5
}

training_params['StoppingCondition'] = { #timeout
    'MaxRuntimeInSeconds' : 86400
}


training_params['HyperParameters'] = {
    "max_depth": "3",
    "eta": "0.05",
    "gamma": "7.46",
    "min_child_weight": "6",
    "subsample": "0.52",
    "objective": "reg:linear",
    "early_stopping_rounds": "10",
    "num_round": "200"
}

training_params['InputDataConfig'] = [
    {
        'ChannelName' : 'train',
        'DataSource' : {
            'S3DataSource' : {
                'S3DataType' : 'S3Prefix',
                'S3Uri' : train_location,
                'S3DataDistributionType' : 'FullyReplicated'
            }
        },
        'ContentType' : 'csv',
        'CompressionType' : 'None'
    }
]

training_job_name = 'train-jo1' + strftime('%Y-%m-%d-%H-%M-%S', gmtime())
training_params['TrainingJobName'] = training_job_name


training_job = session.sagemaker_client.create_training_job(**training_params)


session.logs_for_job(training_job_name, wait=True)



	get_image_uri(region, 'xgboost', '0.90-1').


2020-07-03 00:52:41 Starting - Starting the training job...
2020-07-03 00:52:43 Starting - Launching requested ML instances.........
2020-07-03 00:54:25 Starting - Preparing the instances for training......
2020-07-03 00:55:29 Downloading - Downloading input data...
2020-07-03 00:56:06 Training - Training image download completed. Training in progress..Arguments: train
[2020-07-03:00:56:07:INFO] Running standalone xgboost training.
[2020-07-03:00:56:07:INFO] Path /opt/ml/input/data/validation does not exist!
[2020-07-03:00:56:07:INFO] File size need to be processed in the node: 320.32mb. Available memory size in the node: 152523.7mb
[2020-07-03:00:56:07:INFO] Determined delimiter of CSV input is ','
[00:56:07] S3DistributionType set as FullyReplicated
[00:56:07] 2161358x7 matrix with 15129506 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[00:56:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[

[00:56:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[50]#011train-rmse:0.46075
[00:56:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[51]#011train-rmse:0.45833
[00:56:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[52]#011train-rmse:0.455949
[00:56:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[53]#011train-rmse:0.453622
[00:56:19] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[54]#011train-rmse:0.451336
[00:56:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[55]#011train-rmse:0.449361
[00:56:20] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[56]#011train-rmse:0.447258
[00:56:20] src/tree/up

[00:56:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[144]#011train-rmse:0.377829
[00:56:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[145]#011train-rmse:0.377519
[00:56:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[146]#011train-rmse:0.377359
[00:56:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[147]#011train-rmse:0.377056
[00:56:39] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[148]#011train-rmse:0.376877
[00:56:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[149]#011train-rmse:0.376664
[00:56:40] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[150]#011train-rmse:0.37654
[00:56:40] src


2020-07-03 00:56:59 Uploading - Uploading generated training model
2020-07-03 00:56:59 Completed - Training job completed
Training seconds: 90
Billable seconds: 90


In [7]:
training_job_info = session.sagemaker_client.describe_training_job(TrainingJobName=training_job_name)

model_artifacts = training_job_info['ModelArtifacts']['S3ModelArtifacts']

model_name = training_job_name + "-model"

primary_container = {
    "Image": container,
    "ModelDataUrl": model_artifacts
}

model_info = session.sagemaker_client.create_model(
                                ModelName = model_name,
                                ExecutionRoleArn = role,
                                PrimaryContainer = primary_container)